<a href="https://colab.research.google.com/github/AbduzZami/ColabNotebooks/blob/main/Deblur_Compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abduzzami/compare-deblur")

print("Path to dataset files:", path)

100%|██████████| 439M/439M [00:08<00:00, 56.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abduzzami/compare-deblur/versions/1


In [2]:
path

'/root/.cache/kagglehub/datasets/abduzzami/compare-deblur/versions/1'

In [3]:
# prompt: check files inside path

import os

print(os.listdir(path))


['Compare']


In [8]:
os.listdir(path+'/Compare/CodeFormer')

['blurred_image83.png',
 'blurred_image178.png',
 'blurred_image66.png',
 'blurred_image55.png',
 'blurred_image200.png',
 'blurred_image85.png',
 'blurred_image40.png',
 'blurred_image111.png',
 'blurred_image126.png',
 'blurred_image100.png',
 'blurred_image15.png',
 'blurred_image106.png',
 'blurred_image75.png',
 'blurred_image159.png',
 'blurred_image96.png',
 'blurred_image88.png',
 'blurred_image153.png',
 'blurred_image17.png',
 'blurred_image124.png',
 'blurred_image82.png',
 'blurred_image26.png',
 'blurred_image10.png',
 'blurred_image44.png',
 'blurred_image45.png',
 'blurred_image46.png',
 'blurred_image167.png',
 'blurred_image152.png',
 'blurred_image187.png',
 'blurred_image131.png',
 'blurred_image4.png',
 'blurred_image177.png',
 'blurred_image30.png',
 'blurred_image199.png',
 'blurred_image163.png',
 'blurred_image147.png',
 'blurred_image151.png',
 'blurred_image138.png',
 'blurred_image101.png',
 'blurred_image166.png',
 'blurred_image148.png',
 'blurred_image22.p

In [11]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim

def compute_metrics(img1, img2):
    # Resize img2 to match img1's dimensions
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    psnr_value = cv2.PSNR(img1, img2)

    # Ensure minimum size for SSIM
    win_size = min(7, min(img1.shape[:2]))

    ssim_value = ssim(img1, img2, channel_axis=-1, data_range=img2.max() - img2.min(), win_size=win_size)
    return psnr_value, ssim_value

# Define folder paths
folders = [path+'/Compare/CodeFormer', path+'/Compare/RestoreFormer', path+'/Compare/RestoreFormerPP', path+'/Compare/Blurred', path+'/Compare/Deblurred', path+'/Compare/GFPGAN']
original_folder = path+'/Compare/Original'

# Get sorted filenames from each folder
filenames_original = sorted(os.listdir(original_folder))
folder_files = {folder: sorted(os.listdir(folder)) for folder in folders}

# Store results
results = []

for idx, filename in enumerate(filenames_original):
    original_path = os.path.join(original_folder, filename)
    if not os.path.isfile(original_path):
        continue

    original_img = cv2.imread(original_path)

    for folder in folders:
        if idx >= len(folder_files[folder]):
            continue

        test_filename = folder_files[folder][idx]
        test_path = os.path.join(folder, test_filename)

        if not os.path.isfile(test_path):
            continue

        test_img = cv2.imread(test_path)

        # Compute SSIM and PSNR
        psnr_value, ssim_value = compute_metrics(original_img, test_img)

        results.append([test_filename, folder, psnr_value, ssim_value])

# Convert results to DataFrame and save to CSV
df = pd.DataFrame(results, columns=['Filename', 'Folder', 'PSNR', 'SSIM'])
df.to_csv('ssim_psnr_results.csv', index=False)

print("Metrics computed and saved to ssim_psnr_results.csv")


Metrics computed and saved to ssim_psnr_results.csv


In [12]:
df.head()

,Filename,Folder,PSNR,SSIM
0,blurred_image1.png,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.647956,0.559026
1,blurred_image1.png,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.385131,0.547389
2,blurred_image1.png,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.131780,0.526189
3,blurred_image1.png,/root/.cache/kagglehub/datasets/abduzzami/comp...,21.977617,0.511091
4,deblurred_image1.png,/root/.cache/kagglehub/datasets/abduzzami/comp...,23.211312,0.535759


In [13]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim

def compute_metrics(img1, img2):
    # Resize img2 to match img1's dimensions
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    psnr_value = cv2.PSNR(img1, img2)

    # Ensure minimum size for SSIM
    win_size = min(7, min(img1.shape[:2]))

    ssim_value = ssim(img1, img2, channel_axis=-1, data_range=img2.max() - img2.min(), win_size=win_size)
    return psnr_value, ssim_value

# Define folder paths
folders = [path+'/Compare/CodeFormer', path+'/Compare/RestoreFormer', path+'/Compare/RestoreFormerPP', path+'/Compare/Blurred', path+'/Compare/Deblurred', path+'/Compare/GFPGAN']
original_folder = path+'/Compare/Original'

# Get sorted filenames from each folder
filenames_original = sorted(os.listdir(original_folder))
folder_files = {folder: sorted(os.listdir(folder)) for folder in folders}

# Store results
results = []
folder_metrics = {folder: {'psnr': [], 'ssim': []} for folder in folders}

for idx, filename in enumerate(filenames_original):
    original_path = os.path.join(original_folder, filename)
    if not os.path.isfile(original_path):
        continue

    original_img = cv2.imread(original_path)

    for folder in folders:
        if idx >= len(folder_files[folder]):
            continue

        test_filename = folder_files[folder][idx]
        test_path = os.path.join(folder, test_filename)

        if not os.path.isfile(test_path):
            continue

        test_img = cv2.imread(test_path)

        # Compute SSIM and PSNR
        psnr_value, ssim_value = compute_metrics(original_img, test_img)

        results.append([test_filename, folder, psnr_value, ssim_value])
        folder_metrics[folder]['psnr'].append(psnr_value)
        folder_metrics[folder]['ssim'].append(ssim_value)

# Convert results to DataFrame and save to CSV
df = pd.DataFrame(results, columns=['Filename', 'Folder', 'PSNR', 'SSIM'])
df.to_csv('ssim_psnr_results.csv', index=False)

# Compute mean PSNR and SSIM for each folder
mean_results = []
for folder, metrics in folder_metrics.items():
    mean_psnr = np.mean(metrics['psnr']) if metrics['psnr'] else 0
    mean_ssim = np.mean(metrics['ssim']) if metrics['ssim'] else 0
    mean_results.append([folder, mean_psnr, mean_ssim])

# Save mean results to CSV
df_mean = pd.DataFrame(mean_results, columns=['Folder', 'Mean_PSNR', 'Mean_SSIM'])
df_mean.to_csv('mean_ssim_psnr_results.csv', index=False)

print("Metrics computed and saved to ssim_psnr_results.csv")
print("Mean metrics computed and saved to mean_ssim_psnr_results.csv")

Metrics computed and saved to ssim_psnr_results.csv
Mean metrics computed and saved to mean_ssim_psnr_results.csv


In [14]:
df_mean

,Folder,Mean_PSNR,Mean_SSIM
0,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.594790,0.597722
1,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.914940,0.615396
2,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.880480,0.616674
3,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.432657,0.590689
4,/root/.cache/kagglehub/datasets/abduzzami/comp...,23.462150,0.682138
5,/root/.cache/kagglehub/datasets/abduzzami/comp...,22.990037,0.625591
